In [1]:
# --- Шаг 0: Импорт необходимых библиотек ---
import pandas as pd
import requests
from io import StringIO
import numpy as np # numpy понадобится для np.nan

print("Шаг 0: Библиотеки импортированы.")

# --- Шаг 1: Загрузка данных с веб-страницы ---
# URL страницы с отозванными IPO
url = "https://stockanalysis.com/ipos/withdrawn/"

# Заголовок User-Agent для имитации запроса из браузера
headers = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/91.0.4472.124 Safari/537.36'
    )
}

try:
    # Выполняем GET-запрос для получения HTML-кода страницы
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status() # Проверка на ошибки (например, 404 или 500)

    # Используем pd.read_html для извлечения таблиц со страницы
    # Оборачиваем HTML-текст в StringIO, чтобы избежать предупреждений
    tables = pd.read_html(StringIO(response.text))
    
    # Первая таблица на странице - та, что нам нужна
    df = tables[0]
    
    print(f"Шаг 1: Данные успешно загружены. Найдено {len(df)} записей.")
    # print("Первые 5 строк загруженных данных:")
    # print(df.head())

except Exception as e:
    print(f"Не удалось загрузить данные. Ошибка: {e}")
    # Если данные не загрузились, дальнейшее выполнение бессмысленно
    df = pd.DataFrame()

if not df.empty:
    # --- Шаг 2: Создание колонки "Company Class" ---
    def get_company_class(name: str) -> str:
        """Классифицирует компанию по ключевым словам в названии."""
        if not isinstance(name, str):
            return 'Other'
        
        name_lower = name.lower()
        
        if 'acquisition corp' in name_lower or 'acquisition corporation' in name_lower:
            return 'Acq.Corp'
        elif 'inc' in name_lower or 'incorporated' in name_lower:
            return 'Inc'
        elif 'group' in name_lower:
            return 'Group'
        # Порядок важен: 'limited' должно идти до 'holdings'
        elif 'ltd' in name_lower or 'limited' in name_lower:
            return 'Limited'
        elif 'holdings' in name_lower:
            return 'Holdings'
        else:
            return 'Other'

    df['Company Class'] = df['Company Name'].apply(get_company_class)
    print("\nШаг 2: Создана колонка 'Company Class'.")
    # print("Распределение по классам компаний:")
    # print(df['Company Class'].value_counts())

    # --- Шаг 3: Создание колонки "Avg. price" ---
    def calculate_avg_price(price_range: str):
        """Рассчитывает среднюю цену из диапазона."""
        if not isinstance(price_range, str) or price_range == '-':
            return np.nan # Используем numpy.nan для пропущенных значений

        price_range_cleaned = price_range.replace('$', '')
        
        try:
            if '-' in price_range_cleaned:
                low, high = map(float, price_range_cleaned.split('-'))
                return (low + high) / 2.0
            else:
                return float(price_range_cleaned)
        except (ValueError, TypeError):
            return np.nan

    df['Avg. price'] = df['Price Range'].apply(calculate_avg_price)
    print("\nШаг 3: Создана колонка 'Avg. price'.")

    # --- Шаг 4: Преобразование колонки "Shares Offered" ---
    df['Shares Offered'] = pd.to_numeric(df['Shares Offered'], errors='coerce')
    print("\nШаг 4: Колонка 'Shares Offered' преобразована в числовой формат.")

    # --- Шаг 5: Создание колонки "Withdrawn Value" ---
    df['Withdrawn Value'] = df['Shares Offered'] * df['Avg. price']
    # Для ответа в миллионах
    df['Withdrawn Value ($M)'] = df['Withdrawn Value'] / 1_000_000
    print("\nШаг 5: Рассчитана стоимость отозванных IPO ('Withdrawn Value').")

    # --- Шаг 6: Группировка и расчет итогов ---
    # Группируем по классу компании и суммируем стоимость в миллионах
    total_withdrawn_by_class = df.groupby('Company Class')['Withdrawn Value ($M)'].sum().sort_values(ascending=False)
    
    # Находим класс с максимальным значением
    top_class = total_withdrawn_by_class.idxmax()
    max_value = total_withdrawn_by_class.max()
    
    print("\nШаг 6: Рассчитана общая стоимость отозванных IPO по каждому классу:")
    print(total_withdrawn_by_class.round(2).to_string())

    # --- Финальный ответ ---
    print("\n" + "="*40)
    print("                ОТВЕТ НА ВОПРОС 1")
    print("="*40)
    print(f"Класс компаний с наибольшей стоимостью отозванных IPO: '{top_class}'")
    print(f"Общая стоимость для этого класса: ${max_value:,.2f} миллионов")
    print("="*40)

Шаг 0: Библиотеки импортированы.
Шаг 1: Данные успешно загружены. Найдено 100 записей.

Шаг 2: Создана колонка 'Company Class'.

Шаг 3: Создана колонка 'Avg. price'.

Шаг 4: Колонка 'Shares Offered' преобразована в числовой формат.

Шаг 5: Рассчитана стоимость отозванных IPO ('Withdrawn Value').

Шаг 6: Рассчитана общая стоимость отозванных IPO по каждому классу:
Company Class
Acq.Corp    4021.00
Inc         2257.16
Other        767.92
Limited      549.73
Holdings      75.00
Group         33.79

                ОТВЕТ НА ВОПРОС 1
Класс компаний с наибольшей стоимостью отозванных IPO: 'Acq.Corp'
Общая стоимость для этого класса: $4,021.00 миллионов
